In [1]:
#Import packages
import pandas as pd
import numpy as np
import glob
import re
import string
import codecs

In [12]:
#Create single parliament_qs dataframe with all question data
li = []

for file in glob.glob('./india_headlines_data_pre/*.csv'):
    data = pd.read_csv(file)
    li.append(data)
    
headline = pd.concat(li, axis = 0, ignore_index = True)

In [13]:
headline

publish_date        headline_category  \
0            20010101               sports.wwe   
1            20010102                bollywood   
2            20010102                  unknown   
3            20010102                  unknown   
4            20010102                  unknown   
5            20010102                  unknown   
6            20010102                  unknown   
7            20010102                  unknown   
8            20010102                  unknown   
9            20010102                  unknown   
10           20010102                  unknown   
11           20010102                  unknown   
12           20010102                  unknown   
13           20010102                  unknown   
14           20010102                  unknown   
15           20010102                  unknown   
16           20010102                  unknown   
17           20010102                  unknown   
18           20010102                  unknown   
19           20010102                  unknown   
20           20010102                  unknown   
21           20010102                  unknown   
22           20010102                  unknown   
23           20010102                  unknown   
24           20010102                  unknown   
25           20010102                  unknown   
26           20010102                  unknown   
27           20010102                  unknown   
28           20010102                  unknown   
29           20010102                  unknown   
...               ...                      ...   
1781923      20140409            tv.news.hindi   
1781924      20140409            tv.news.hindi   
1781925      20140409          tv.news.kannada   
1781926      20140409            tv.news.tamil   
1781927      20140409                  unknown   
1781928      20140409                  unknown   
1781929      20140409                  unknown   
1781930      20140409                  unknown   
1781931      20140409              world.china   
1781932      20140409      world.rest-of-world   
1781933      20140409                 world.us   
1781934      20140410                bangalore   
1781935      20140410  business.india-business   
1781936      20140410  business.india-business   
1781937      20140410  business.india-business   
1781938      20140410  business.india-business   
1781939      20140410  business.india-business   
1781940      20140410  business.india-business   
1781941      20140410  business.india-business   
1781942      20140410  business.india-business   
1781943      20140410  business.india-business   
1781944      20140410  business.india-business   
1781945      20140410  business.india-business   
1781946      20140410  business.india-business   
1781947      20140410  business.india-business   
1781948      20140410  business.india-business   
1781949      20140410  business.india-business   
1781950      20140410  business.india-business   
1781951      20140410  business.india-business   
1781952      20140410  business.india-business   

                                             headline_text  
0        win over cena satisfying but defeating underta...  
1                                              Raju Chacha  
2        Status quo will not be disturbed at Ayodhya; s...  
3                      Fissures in Hurriyat over Pak visit  
4                    America's unwanted heading for India?  
5                       For bigwigs; it is destination Goa  
6                     Extra buses to clear tourist traffic  
7              Dilute the power of transfers; says Riberio  
8                        Focus shifts to teaching of Hindi  
9                     IT will become compulsory in schools  
10           Move to stop freedom fighters' pension flayed  
11       Gilani claims he applied for passport 2 years ago  
12                          Parivar dismisses PM's warning  
13                   India; Pak exchange lists of N-pl

In [14]:
#Code borrowed and adapted from George Chen, Carnegie Mellon University#
#Define function to remove punctuation and whitespace, and lowercase all text
def makeWordList(str_object):
    
    corpus_text = str(str_object)
    
    for c in string.punctuation:
        corpus_text = corpus_text.replace(c, "")  # -- (1)
    
    text = re.sub(r'\S*\d\S*','',corpus_text) # -- (2)
    text = re.sub(r'[^\w\s]','',text)         # -- (3)
    
    text = text.lower().split()           # -- (4)         
    
    li = []
    for token in text:
        li.append(token)

    return " ".join(li)

In [18]:
#Process the questions
processed_headline = []

for str_object in list(headline["headline_text"]):
    processed_headline.append(makeWordList(str_object))

In [22]:
#Use TfidfVectorizer to transform parliamentary questions
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, stop_words="english", max_df=0.8)
headline_fit = vectorizer.fit(processed_headline)
X_headline = vectorizer.fit_transform(processed_headline).toarray()

MemoryError: 

In [21]:
#!Time-consuming!#
#Create topics using LDA
num_topics = 25

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', random_state=0)
lda.fit(X_headline)

NameError: name 'X_headline' is not defined

In [ ]:
#Display top 10 words from each topic
words = list(headline_fit.vocabulary_)
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])
num_top_words = 10

print('Displaying the top 10 words per topic and their probabilities within the topic...')
print()

for topic_idx in range(num_topics):
    print('[Topic ', topic_idx, ']', sep='')
    sort_indices = np.argsort(-topic_word_distributions[topic_idx])
    for rank in range(num_top_words):
        word_idx = sort_indices[rank]
        print(words[word_idx], ':', topic_word_distributions[topic_idx, word_idx])
    print()

In [23]:
lda.components_

AttributeError: 'LatentDirichletAllocation' object has no attribute 'components_'